In [1]:
import torch
import numpy as np
import os
import glob
from brainscore_vision import load_model

/home/seo/miniconda3/envs/myvenv/lib/python3.11/site-packages/brainscore_core/metrics/__init__.py:16: FutureWarning: xarray subclass Score should explicitly define __slots__
  class Score(DataAssembly):


  Using cached torchvision-0.20.1-cp311-cp311-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached albumentations-1.4.22-py3-none-any.whl.metadata (33 kB)
  Using cached timm-1.0.12-py3-none-any.whl.metadata (51 kB)
  Using cached pydantic-2.10.3-py3-none-any.whl.metadata (172 kB)
  Using cached albucore-0.0.21-py3-none-any.whl.metadata (5.3 kB)
  Using cached eval_type_backport-0.2.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached stringzilla-3.11.1-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_28_x86_64.whl.metadata (80 kB)
  Using cached simsimd-6.2.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (66 kB)
  Using cached huggingface_hub-0.26.5-py3-none-any.whl.metadata (13 kB)
  Using cached safetensors-0.4.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metad

In [ ]:
model = load_model('deit_base_imagenet_full_seed-0')
act_model = model.activations_model

In [4]:
layers = ['blocks.3.mlp.fc1', 'blocks.8.norm2', 'blocks.3.mlp.act', 'blocks.9.norm2']

In [ ]:
imagenet = f'{ os.getcwd() }/data/imagenet/val'

In [5]:
stimuli = glob.glob(os.path.join(imagenet, '**'), recursive=True)
stimuli = [ stimulus for stimulus in stimuli if os.path.isfile(stimulus) ]

In [ ]:
output = f'{ os.getcwd() }/data/output/deit-base'

In [7]:
# Set batch size for processing
batch_size = 1000

# Initialize counter for file naming
counter = 1

# Loop through the stimuli in batches
for batch_start in range(0, len(stimuli), batch_size):
  batch = stimuli[batch_start:batch_start + batch_size]
  model_extraction = act_model(batch, layers)
  layers = model_extraction['layer'].values
  features_by_layer = {
    layer: model_extraction.sel(neuroid=model_extraction['layer'] == layer).data
    for layer in layers
  }
  npz_data = {
    layer_name: features 
    for layer_name, features in features_by_layer.items()
  }
  np.savez(os.path.join(output, '{}-{}.npz'.format(model.identifier, counter)), **npz_data)
  counter += 1

activations:   0%|          | 0/1024 [00:00<?, ?it/s]

: 